In [1]:
import os

In [2]:
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project\\research'

In [3]:
os.chdir("../")
%pwd

'd:\\Data_analytics_new\\FSDS\\learning\\DS_Project\\DS-Project'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelTrainingConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    alpha: float
    l1_ratio: float
    target_column: str

In [5]:
from src.ds_project.constants import *
from src.ds_project.utils.common import read_yaml, create_directories, save_bin

In [12]:
class ConfigurationManager:
    def __init__(self, config_filepath=CONFIG_FILE_PATH,
                 params_file_path=PARAMS_FILE_PATH,
                 schema_file_path=SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_file_path)
        self.schema = read_yaml(schema_file_path)

        #print(self.config.artifacts_root)

        create_directories(self.config.artifacts_root)

    def get_model_trainer_config(self)-> ModelTrainingConfig:
        config=self.config.model_trainer
        params=self.params.ElasticNet
        schema=self.schema.TARGET_COLUMN
        create_directories(config.root_dir)

        model_trainer_config=ModelTrainingConfig(
            root_dir=config.root_dir,
            train_data_path=config.train_file_path,
            test_data_path=config.test_file_path,
            model_name=config.model_name,
            alpha=params.alpha,
            l1_ratio=params.l1_ratio,
            target_column=schema.name,
        )
        return model_trainer_config

In [13]:
import pandas as pd
import os
from src.ds_project import logger
from sklearn.linear_model import ElasticNet
import joblib

In [14]:
class ModelTrainer:
    def __init__(self, config: ModelTrainingConfig):
        self.config=config

        ### Add any techniques here

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)

        train_x = train_data.drop([self.config.target_column], axis=1)
        test_x = test_data.drop([self.config.target_column],axis=1)
        train_y = train_data[[self.config.target_column]]
        test_y = test_data[[self.config.target_column]]

        lr=ElasticNet(alpha=self.config.alpha, l1_ratio=self.config.l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        joblib.dump(lr, os.path.join(self.config.root_dir, self.config.model_name))

      

In [15]:
try:
    config=ConfigurationManager()
    model_trainer_config = config.get_model_trainer_config()
    model_trainer = ModelTrainer(config=model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e


<_io.TextIOWrapper name='config\\config.yaml' mode='r' encoding='utf-8'>
{'artifacts_root': ['artifacts'], 'data_ingestion': {'root_dir': ['artifacts/data_ingestion'], 'source_URL': 'https://raw.githubusercontent.com/dushyantverma22/data_set/main/WineQT.csv', 'local_data_file': 'artifacts/data_ingestion/WineQT.csv', 'unzip_dir': 'artifacts/data_ingestion'}, 'data_validation': {'root_dir': ['artifacts/data_validation'], 'unzip_data_dir': 'artifacts/data_ingestion/WineQT.csv', 'STATUS_FILE': 'artifacts/data_validation/status.txt'}, 'data_transformation': {'root_dir': 'artifacts/data_transforamtion', 'data_path': 'artifacts/data_ingestion/WineQT.csv'}, 'model_trainer': {'root_dir': 'artifacts/model_trainer', 'train_file_path': 'artifacts/data_transforamtion/train.csv', 'test_file_path': 'artifacts/data_transforamtion/test.csv', 'model_name': 'model.joblib'}}
[2024-10-22 00:43:20,687: INFO: common: yaml file: config\config.yaml loaded successfully]
<_io.TextIOWrapper name='params.yaml' mod